Process several CTD profiles, add nutrient data and merge to just one table. 

In [124]:
import numpy as np
import pandas as pd

File names:

In [125]:
filenames = ['in2024_t01_001.cnv', 'in2024_t01_002.cnv', 'in2024_t01_003.cnv', 'in2024_t01_004.cnv', 
             'in2024_t01_005.cnv', 'in2024_t01_006.cnv', 'in2024_t01_007.cnv']

The loop below iterates throught the files, processes the data, and then stacks them vertically.

In [129]:
for i in range(len(filenames)):
    if i == 0:
        # Open file and assign column names
        column_names = ['t090C', 'c0Sm', 'prDM', 't190C', 'c1Sm', 'sbeox0MmL', 'sbeox1MmL', 
                        'sal00', 'sal11', 'scan', 'timeS', 'pumps', 'latitude', 'longitude', 
                        'flECO_AFL', 'turbWETbb0', 'altM', 'bpos', 'nbf', 'par', 'wetCDOM', 'CStarTr0', 'flag']
        data = pd.read_table(filenames[i], skiprows = 345, sep='\\s+', names = column_names)
        # Remove data with pressure less than 12
        data = data.loc[data['prDM'] > 12]
        # Find the maximum pressure and use it to retain only the downcast
        max_pressure_idx = np.argmax(data['prDM'])
        data = data[:max_pressure_idx]
        # Group the pressure column and average to sample down to integers
        data['PressureInterval'] = data['prDM'].apply(lambda x: int(x))
        data = data.groupby('PressureInterval').mean().reset_index()
        data = data.drop(columns = 'prDM')
        # Add CTD number
        data['CTD_No'] = i+1
    else:
        # Open file and assign column names
        column_names = ['t090C', 'c0Sm', 'prDM', 't190C', 'c1Sm', 'sbeox0MmL', 'sbeox1MmL', 
                        'sal00', 'sal11', 'scan', 'timeS', 'pumps', 'latitude', 'longitude', 
                        'flECO_AFL', 'turbWETbb0', 'altM', 'bpos', 'nbf', 'par', 'wetCDOM', 'CStarTr0', 'flag']
        ds = pd.read_table(filenames[i], skiprows = 345, sep='\\s+', names = column_names)
        # Remove ds with pressure less than 12
        ds = ds.loc[ds['prDM'] > 12]
        # Find the maximum pressure and use it to retain only the downcast
        max_pressure_idx = np.argmax(ds['prDM'])
        ds = ds[:max_pressure_idx]
        # Group the pressure column and average to sample down to integers
        ds['PressureInterval'] = ds['prDM'].apply(lambda x: int(x))
        ds = ds.groupby('PressureInterval').mean().reset_index()
        ds = ds.drop(columns = 'prDM')
        ds['CTD_No'] = i+1
        data = pd.concat([data, ds], axis = 0)

Now we will import the nutrient data and use the rounded pressure column and CTD number to add nutrient info to our merged table.

In [130]:
nuts = pd.read_csv('Nuts_sorted_with_depth_v2.csv')
nuts

,Dep_No_,Bottle,pressureDb,NH4_uM,NH4_flag,NOx_uM,NOx_flag,Nitrite_uM,Nitrite_flag,PO4_uM,PO4_flag,Silicate_uM,Silicate_flag
0,1,4,147,0.03,Good,1.06,Good,0.072,Good,0.18,Good,1.7,Good
1,1,11,110,0.02,Good,1.23,Good,0.055,Good,0.19,Good,2.1,Good
2,1,18,87,0.06,Good,0.19,Good,0.083,Good,0.11,Good,1.6,Good
3,1,22,40,0.03,Good,0.02,Good,0.018,Good,0.07,Good,1.2,Good
4,1,23,20,0.02,Good,0.02,Good,0.016,Good,0.06,Good,1.1,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,7,26,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,7,30,24,0.14,Good,0.14,Good,0.042,Good,0.19,Good,-0.2,Good
111,7,31,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,7,32,10,0.15,Good,0.13,Good,0.038,Good,0.19,Good,-0.2,Good


The nutrients data has `Dep_No` instead of `CTD_No`. Let's replace that. Same for the pressures

In [131]:
nuts = nuts.rename(columns = {'Dep_No_':'CTD_No', 'pressureDb':'PressureInterval'})
merged_table = pd.merge(data, nuts, on = ['CTD_No', 'PressureInterval'], how = 'outer')

Add chlorophyll data:

In [132]:
chlaData = pd.read_csv('In2024_T01_Chla_CTD.csv');
chlaData = chlaData.rename(columns = {'Bottle_No':'Bottle'})

In [133]:
merged_table = pd.merge(merged_table, chlaData, on = ['CTD_No', 'Bottle'], how = 'outer')

Finally, we want to replace the flags by numeric values. If it's good, we will replace with 0, and Suspect with 4.

In [135]:
for column in ['NH4_flag', 'NOx_flag', 'Nitrite_flag', 'PO4_flag', 'Silicate_flag']:
    idx = np.where(merged_table[column] == 'Good')[0]
    merged_table.loc[idx, column] = 0
    idx = np.where(merged_table[column] == 'Suspect')[0]
    merged_table.loc[idx, column] = 4

In [136]:
# Sort the table
merged_table.sort_values(['CTD_No', 'PressureInterval'], ascending = [True, True])

,PressureInterval,t090C,c0Sm,t190C,c1Sm,sbeox0MmL,sbeox1MmL,sal00,sal11,scan,...,NOx_uM,NOx_flag,Nitrite_uM,Nitrite_flag,PO4_uM,PO4_flag,Silicate_uM,Silicate_flag,Chla,Flag_ODV
5,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.02,0,0.016,0,0.07,0,1.2,0,0.116,0.0
6,12,21.515705,5.046881,21.520187,5.047049,211.406175,214.326250,35.794820,35.792435,15812.475000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,13,21.519047,5.047248,21.521344,5.047171,211.692313,214.494656,35.794625,35.792084,15848.531250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,14,21.516238,5.047015,21.513850,5.046500,212.040154,213.910598,35.794679,35.792569,15922.991453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,15,21.483911,5.043525,21.492678,5.044185,211.552435,214.499804,35.793507,35.791467,16004.521739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,2208,2.127784,3.162202,2.128597,3.162278,165.136226,170.195857,34.730946,34.731017,78459.070199,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20527,2209,2.128281,3.162285,2.129088,3.162361,165.114539,170.205013,34.730937,34.731014,79299.613924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20528,2210,2.128632,3.162356,2.129409,3.162429,165.163920,170.159095,34.730947,34.731006,80438.093775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20529,2211,2.128968,3.162428,2.129706,3.162499,165.163860,170.217643,34.730946,34.731000,82236.554969,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_table.to_csv('merged_data_file.csv')